<h1>VIT Testing - tamncheese Jason Kahei Tam<h1>


Import

In [3]:
import torch
import torch.nn as nn
import os
import io
from torch.utils.data import DataLoader, Dataset
from transformers import ViTImageProcessor, ViTForImageClassification, AutoFeatureExtractor, AutoModel, \
    AutoModelForImageClassification, AutoImageProcessor
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
from torchvision import datasets, transforms
import pandas as pd


CPU or CUDA


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

Prepare Dataset

In [7]:
file_list = []
for root, dirs, files, in os.walk("testing"):
    for file in files:
        file_list.append(file)
file_list

train_df = pd.read_csv("_dataset_trial/metadata_trial.csv")
n_labels = train_df["species"].nunique()
print(f"Number of labels: {n_labels}")



Number of labels: 1577


In [8]:


class FungiDataset(Dataset):
    def __init__(self, df, extractor, transform = None, local_filepath = None):
        self.df = df
        self.transform = transform
        self.extractor = extractor
        self.local_filepath = local_filepath

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label = self.df.iloc[idx]["species"]
        if self.local_filepath:
            img_path = self.local_filepath + self.df["image_path"].values[idx].replace("jpg", "JPG")
            try:
                # Load Images (OpenCV)
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            except:
                print(f"Missing image: {img_path}")
                image = np.random.uniform(-1,1,size=(299,299,3)).astype(np.float32)
        else:
            image = Image.open(io.BytesIO(self.df.data.values[idx]))
            image = np.array(image)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented["image"]

        image = self.extractor(images=image, return_tensors="pt")["pixel_values"].squeeze(0)
        return image, torch.tensor(label, dtype = torch.long)



Configure DINOv2


In [9]:
extractor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModelForImageClassification.from_pretrained('facebook/dinov2-base', num_labels=n_labels)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Frreze Backbone

In [10]:
for param in model.base_model.parameters():
    param.requires_grad = False

print("Trainable Layers")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name} is trainable")

Trainable Layers
classifier.weight is trainable
classifier.bias is trainable


Load Data


In [11]:
img_dir = "training_laptop"
train_dataset = FungiDataset(train_df, extractor, img_dir)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

Train the Model


In [1]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = 0.00001)
criterion = nn.CrossEntropyLoss()

epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)
    print(f"Training Loss: {avg_train_loss} for epoch {epoch + 1}/{epochs}")



NameError: name 'torch' is not defined